------------- FUNCTION EXPLANATIONS -------------<br/>
break  : stop the loop to the first iteration <br/>
split : it'll take any texts and make 2 blocks with it (before and after)<br/>
df : shortcut in order to access to the excel file (such pd for pandas)<br/>
loc[n] : locate the first line

------------- OBJECTIVES OF THE SESSIONS -------------<br/>
1- find the iso codes<br/>
2- put it inside the code<br/>
3- Displays the last gc tax for each country<br/>

In [325]:
import pandas as pd
import requests as rq

In [326]:
df = pd.read_excel('Data-sheet-STUDENTS.xlsx',header=2,index_col=0)
#df['Country (according to CN Gov):']

In [ ]:
iso_codes = {}    #create a table that will contain all the iso_codes of each countries

In [ ]:
for index, name in df['Country (according to CN Gov):'].items():  #create the loop to only have the names of the countries
    if name in iso_codes:
        continue    #means that if the name already exists then it'll pass the step

    cleaned_name = name.split('(')[0].strip()    #remove the error of France (Not BRI) / Thailiand / USA...
    if cleaned_name == 'Thailiand':
        cleaned_name = 'Thailand'
    if cleaned_name == 'The United Arab Emirates':
        cleaned_name = 'United Arab Emirates'
    
        
    url = 'https://restcountries.eu/rest/v2/name/' + cleaned_name   #don't forget to change into cleaned_name
    r = rq.get(url)  #create the fonction request to obtain the isocodes
    if r.status_code == 200:
        data = r.json() #check the status code if 200 it's success
        country_code = data[0]['alpha3Code']   #enter inside the table and go to the first column and then to the parameter
        if country_code == 'IOT':
            country_code = 'IND'
        iso_codes[name] = country_code   #put the code inside the table
    else:
        print('ERROR',r.status_code,r.text)
        print(name)

In [327]:
df['isocode'] = df['Country (according to CN Gov):'].apply(lambda x: iso_codes[x])

In [328]:
df[['isocode','Country (according to CN Gov):']]

,isocode,Country (according to CN Gov):
1,AFG,Afghanistan
2,ALB,Albania
3,ARM,Armenia
4,AZE,Azerbaijan
5,BHR,Bahrain
6,BGD,Bangladesh
7,BLR,Belarus
8,BTN,Bhutan
9,BIH,Bosnia and Herzegovina
10,BRN,Brunei


In [ ]:
#Get the indicator for one country
"""  
url_tax = 'http://api.worldbank.org/v2/country/brn/indicator/GC.TAX.IMPT.ZS?format=json&MRV=1' 

r = rq.get(url_tax) 
if r.status_code == 200:
    data = r.json()
    if data[1] is None:
        print('yes')
    else:
        print('no')
    #tax_value = data[1][0]['value']  
    #print(tax_value)
else:
    print('ERROR',r.status_code,r.text)
    print(name)
"""

In [ ]:
taxValues = {} 

In [334]:
#Get the indicator for all the countries

for index, iso in df['isocode'].items():
    
    url_tax = 'http://api.worldbank.org/v2/country/'+iso+'/indicator/GC.TAX.IMPT.ZS?format=json&MRV=1' 
    
    r = rq.get(url_tax) 
    if r.status_code == 200:
        data = r.json() 
        if data[1] is None:    #prevent the problem of countries without any tax value
            tax_country = 0
        else:
            tax_country = data[1][0]['value']  
        tax_values[iso] = tax_country
    else:
        print('ERROR',r.status_code,r.text)
        print(iso)     

In [335]:
#create the first table with all the tax without making the link with the real column

df['Tax Values'] = df['isocode'].apply(lambda x: tax_values[x])
df['Tax Values']

1     35.203141
2      2.216459
3      6.275395
4      7.571847
5     27.386091
6     28.923008
7      8.200619
8      2.840501
9      0.000155
10     0.000000
11     0.068860
12    13.542016
13     3.894341
14    -0.037631
15     0.819567
16     4.779919
17     6.849002
18     0.039429
19    -0.023245
20     0.757021
21     0.021249
22    -0.000145
23    12.808273
24     2.568975
25    20.696450
26    14.105211
27     0.962753
28     7.006193
29     5.299519
30    66.806723
        ...    
39     3.423233
40    13.502166
41     0.000000
42     9.003222
43    17.770038
44     0.000000
45     0.000000
46    20.357608
47     0.128546
48     0.000000
49     0.035332
50     6.217626
51    23.017305
52     5.062219
53     0.071508
54     0.000066
55     0.808280
56    22.054891
57     0.000000
58    15.943747
59     3.952757
60     0.381747
61     2.504523
62     0.000000
63     0.047747
64     2.642594
65     3.997681
66     5.186860
67     0.000000
68     0.000000
Name: Tax Values, Length

In [336]:
df['IMF WB\nCustoms and other import duties (% of tax revenue) 2016']

1     35.203141
2      2.216459
3      6.275395
4      7.571847
5     27.386091
6     28.923008
7      8.200619
8      2.840501
9      0.000155
10     0.000000
11     0.068860
12    15.785000
13     3.894341
14    -0.037631
15     0.819567
16     4.779919
17     6.849002
18     0.039429
19    -0.023245
20     0.757021
21     0.021249
22    -0.000145
23    12.808273
24     2.568975
25    20.696450
26    14.105211
27     0.962753
28     7.006193
29     5.299519
30    66.806723
        ...    
39     3.423233
40    13.502166
41     0.000000
42     9.003222
43    17.770038
44     0.000000
45     0.000000
46    20.357608
47     0.128546
48     0.000000
49     0.035332
50     6.217626
51    23.017305
52     5.062219
53     0.071508
54     0.000066
55     0.808280
56    22.054891
57     0.000000
58    15.943747
59     3.952757
60     0.381747
61     2.504523
62     0.000000
63     0.047747
64     2.642594
65     3.997681
66     5.186860
67     0.000000
68     0.000000
Name: IMF WB\nCustoms an

In [337]:
#make a loop that will look if the cell is empty or not
#if it's empty it'll had the value from the new table if it's not it'll let it like it is

for index, tax in df['IMF WB\nCustoms and other import duties (% of tax revenue) 2016'].items():
    if pd.isnull(df.loc[index,'IMF WB\nCustoms and other import duties (% of tax revenue) 2016']):
        df.loc[index,'IMF WB\nCustoms and other import duties (% of tax revenue) 2016'] = df.at[index,'Tax Values']

In [338]:
df['IMF WB\nCustoms and other import duties (% of tax revenue) 2016']

1     35.203141
2      2.216459
3      6.275395
4      7.571847
5     27.386091
6     28.923008
7      8.200619
8      2.840501
9      0.000155
10     0.000000
11     0.068860
12    15.785000
13     3.894341
14    -0.037631
15     0.819567
16     4.779919
17     6.849002
18     0.039429
19    -0.023245
20     0.757021
21     0.021249
22    -0.000145
23    12.808273
24     2.568975
25    20.696450
26    14.105211
27     0.962753
28     7.006193
29     5.299519
30    66.806723
        ...    
39     3.423233
40    13.502166
41     0.000000
42     9.003222
43    17.770038
44     0.000000
45     0.000000
46    20.357608
47     0.128546
48     0.000000
49     0.035332
50     6.217626
51    23.017305
52     5.062219
53     0.071508
54     0.000066
55     0.808280
56    22.054891
57     0.000000
58    15.943747
59     3.952757
60     0.381747
61     2.504523
62     0.000000
63     0.047747
64     2.642594
65     3.997681
66     5.186860
67     0.000000
68     0.000000
Name: IMF WB\nCustoms an